In [1]:
#!pip install -e git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper
#!pip install --upgrade google-cloud-bigquery

In [2]:
import os
import bq_helper
from bq_helper import BigQueryHelper
import pandas as pd
%run country-boundingbox.ipynb

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Data-mining-afa11af25388.json"
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
noaa = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="ghcn_d")

In [50]:
def weatherDataByCountry(countryID):
    coordinates = country_bounding_boxes[countryID][1]
    lonMin = str(coordinates[0])
    latMin = str(coordinates[1])
    lonMax = str(coordinates[2])
    latMax = str(coordinates[3])
    weatherData = pd.DataFrame()
    for i in range(2000,2015):
        query = """SELECT
          stations.id,
          weather.date,
          weather.element,
          weather.value
        FROM
          `bigquery-public-data.ghcn_d.ghcnd_"""+str(i)+"""` AS weather INNER JOIN `bigquery-public-data.ghcn_d.ghcnd_stations` AS stations ON 
          weather.id = stations.id
        WHERE
          weather.element = 'TAVG'
          AND stations.latitude >"""+latMin+"""
          AND stations.latitude <"""+latMax+"""
          AND stations.longitude >"""+lonMin+"""
          AND stations.longitude <"""+lonMax+"""
        ORDER BY
            weather.date ASC;
                """
        response2 = noaa.query_to_pandas_safe(query, max_gb_scanned=10)
        response2.dropna()
        weatherData = weatherData.append(response2)
    return weatherData

dfWeather= weatherDataByCountry('HN')
print(dfWeather.size)
print(dfWeather.head(10))

104948
            id        date element  value
0  HO000078711  2000-01-01    TAVG  264.0
1  HOM00078708  2000-01-01    TAVG  234.0
2  HO000078720  2000-01-01    TAVG  192.0
3  HOM00078705  2000-01-01    TAVG  215.0
4  HOM00078705  2000-01-02    TAVG  237.0
5  HO000078720  2000-01-02    TAVG  205.0
6  HOM00078708  2000-01-02    TAVG  238.0
7  HO000078711  2000-01-02    TAVG  270.0
8  HO000078711  2000-01-03    TAVG  269.0
9  HO000078720  2000-01-03    TAVG  193.0


In [54]:
def closestStationToLoc(lat, long):
    query = """SELECT weather.id, POWER(POWER(weather.latitude - """+str(lat)+""",2) + POWER(weather.longitude - """+str(long)+""",2), 1/2) AS distance, weather.latitude, weather.longitude
    FROM
      `bigquery-public-data.ghcn_d.ghcnd_stations` AS weather
    ORDER BY
        distance ASC
    LIMIT 10;
            """
    response2 = noaa.query_to_pandas_safe(query, max_gb_scanned=10)
    print(response2)
    return response2['id'].iloc[0]

def weatherDataForStationID(ID):
    weatherData = pd.DataFrame()
    for i in range(2000,2015):
        query = """SELECT
          weather.date,
          weather.value
        FROM
          `bigquery-public-data.ghcn_d.ghcnd_"""+str(i)+"""` AS weather
        WHERE
            weather.element = 'TAVG'
            AND weather.id = '"""+ID+"""'
        ORDER BY
            weather.date ASC;
                """
        response2 = noaa.query_to_pandas_safe(query, max_gb_scanned=10)
        response2.dropna()
        weatherData = weatherData.append(response2)
    return weatherData

stationID = closestStationToLoc(15.2000, -86.2419)
dfWeather= weatherDataForStationID(stationID)
print(dfWeather.size)
print(dfWeather.head(10))

            id  distance  latitude  longitude
0  HOM00078705  0.816828   15.7420    -86.853
1  HOW00011806  0.867297   16.0167    -85.950
2  HO000078720  1.507753   14.0500    -87.217
3  HOM00078708  1.701020   15.4530    -87.924
4  HO000078724  2.169227   13.2300    -87.150
5  NU000055002  2.282180   12.9500    -85.860
6  NU000055016  2.326082   12.8800    -86.410
7  NU000055029  2.379013   12.9300    -85.530
8  HO000078711  2.441959   15.2170    -83.800
9  ES004881415  2.453357   13.3300    -87.830
10712
         date  value
0  2000-01-01  215.0
1  2000-01-02  237.0
2  2000-01-03  242.0
3  2000-01-04  241.0
4  2000-01-05  242.0
5  2000-01-06  222.0
6  2000-01-07  222.0
7  2000-01-08  219.0
8  2000-01-09  225.0
9  2000-01-10  268.0
